In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("./instrumentation-candidates-pdfbox.csv")
df.shape

(137, 15)

In [3]:
df.head()

,visibility,parent-FQN,method-name,param-list,return-type,local-variables,conditionals,multiple-statements,loops,parameters,returns,switches,ifs,static,returns-primitives
0,public,org.apache.fontbox.ttf.VerticalMetricsTable,getAdvanceHeight,int,int,False,False,False,False,True,True,False,True,False,True
1,public,org.apache.fontbox.util.BoundingBox,contains,"float, float",boolean,False,False,False,False,True,True,False,False,False,True
2,private,org.apache.pdfbox.pdmodel.encryption.AccessPer...,isPermissionBitOn,int,boolean,False,False,False,False,True,True,False,False,False,True
3,public,org.apache.pdfbox.util.SmallMap,size,NaN,int,False,True,False,False,False,True,False,False,False,True
4,private,org.apache.pdfbox.tools.PDFText2HTML$FontState,openTag,java.lang.String,java.lang.String,False,False,False,False,True,True,False,False,False,False


In [4]:
count = 0;
aspects = [];
base_path = "/home/user/dev/pankti-pdfbox-plugin/src/main/java/se/kth/castor/instrumentation/plugins/PDFBoxAspect";
template_file_path = base_path + str(0) + ".java";
for index, row in df.iterrows():
    with open(template_file_path) as template:
        count += 1;
        aspect_string = "\"se.kth.castor.instrumentation.plugin.PDFBoxAspect" + str(count) + "\"";
        aspects.append(aspect_string);
        new_file_path = base_path + str(count) + ".java";
        with open(new_file_path, "w") as f:
            for line in template:
                if ("public class PDFBoxAspect0" in line):
                    line = line.replace("0", str(count));
                if ("@Pointcut(className =" in line):
                    line = re.sub(r"=\s\"(.+)\",", "= \"" + row['parent-FQN'] + "\",", line);
                if ("methodName = " in line):
                    line = re.sub(r"=\s\"(.+)\",", "= \"" + row['method-name'] + "\",", line);
                if ("methodParameterTypes = " in line):
                    if (pd.isnull(row['param-list'])):
                        paramList = "";
                    else:
                        paramList = str(row['param-list']);
                        paramList = re.sub(r"([a-zA-Z0-9\$\.\[\]]+)", "\"" + r"\1" + "\"", paramList);
                    line = re.sub(r"=\s{.*},", "= {" + paramList + "},", line);
                if ("timerName = " in line):
                    line = re.sub(r"=\s\".+\"\)", "= \"" + row['parent-FQN'] + "-" + row['method-name'] + "\")", line);
                if ("int COUNT = " in line):
                    line = re.sub(r"=\s\d+;", "= " + str(count) + ";", line);
                f.write(line)

In [5]:
print(len(aspects))

137


In [6]:
plugin_json_path = "/home/user/dev/pankti-pdfbox-plugin/src/main/resources/META-INF/glowroot.plugin.json";
index = 0
aspect_lines = "";
for i in range(len(aspects)):
    aspect_lines += aspects[i];
    if i != (len(aspects)-1):
        aspect_lines += ",\n";
with open(plugin_json_path, "r") as json_file:
    for num, line in enumerate(json_file, 1):
        if "aspects" in line:
            index = num
with open(plugin_json_path, "r+") as json_file:
    contents = json_file.readlines()
    contents.insert(index, aspect_lines)
    json_file.seek(0)
    json_file.writelines(contents)